In [20]:
import pandas as pd
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



In [21]:
df = pd.read_csv('data/stud.csv')
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [28]:
X = df.drop(['math_score'],axis=1)
y=df['math_score']

In [29]:
numeric_features = X.select_dtypes(exclude='object').columns
categorical_features = X.select_dtypes(include='object').columns

In [30]:
#converting categorical features into numeric features
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

num_transformer = StandardScaler()
oh_transformer  = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder",oh_transformer, categorical_features),
        ("StandarScaler",num_transformer,numeric_features)
    ]
)

In [31]:
X = preprocessor.fit_transform(X)

In [32]:
X.shape

(1000, 19)

In [33]:
#split the data into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)


In [43]:
#create evaluation function to give all performannce metrics
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
def evaluate(y_true,y_pred):
    mae = mean_absolute_error(y_true,y_pred)
    mse = mean_squared_error(y_true,y_pred)
    rmse = np.sqrt(mse)
    r_square_score = r2_score(y_true,y_pred)

    return mae,rmse,r_square_score


In [51]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

models = {
    "LinearRegression" : LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "KNN" : KNeighborsRegressor(),
    "DecisionTree": DecisionTreeRegressor(),
    "Random_Forest_Regressor": RandomForestRegressor(),
    "XGBRegressor" : XGBRegressor(),
    "CatBoostRegressor": CatBoostRegressor(verbose=False),
    "AdaBoostRegressor":AdaBoostRegressor()
}

model_list = []
r2_score_list = []

for k,v in models.items():
    model = models[k]
    model.fit(X_train,y_train)

    model_list.append(k)
    #make predictions on train and test data
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    #evaluate train and test data
    mae_train, rmse_train, r2_score_train = evaluate(y_train,y_train_pred)

    mae_test, rmse_test, r2_score_test = evaluate(y_test,y_test_pred)

    print("*******************"+k+"**************************")

    print("Model Performance for training:")
    print("\t1.Mean absolute error: ",mae_train)
    print("\t2.Root mean squared error: ",rmse_train)
    print("\t3.R2_score: ",r2_score_train)

    print("-------------------------------------------------")

    print("Model Performance for testing:")
    print("\t1.Mean absolute error: ",mae_test)
    print("\t2.Root mean squared error: ",rmse_test)
    print("\t3.R2_score: ",r2_score_test)
    r2_score_list.append(r2_score_test)
    print("\n\n")


*******************LinearRegression**************************
Model Performance for training:
	1.Mean absolute error:  4.238303571428571
	2.Root mean squared error:  5.27494862608277
	3.R2_score:  0.8849170560038007
-------------------------------------------------
Model Performance for testing:
	1.Mean absolute error:  4.3990625
	2.Root mean squared error:  5.600289328230804
	3.R2_score:  0.8442541089216598



*******************Lasso**************************
Model Performance for training:
	1.Mean absolute error:  5.2173385944631825
	2.Root mean squared error:  6.598353168045297
	3.R2_score:  0.8199282403858398
-------------------------------------------------
Model Performance for testing:
	1.Mean absolute error:  5.193900873325009
	2.Root mean squared error:  6.595373314939616
	3.R2_score:  0.7839897128777309



*******************Ridge**************************
Model Performance for training:
	1.Mean absolute error:  4.232224071225289
	2.Root mean squared error:  5.26680790483571

In [57]:
pd.DataFrame(list(zip(model_list,r2_score_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)

,Model Name,R2_Score
2,Ridge,0.847221
0,LinearRegression,0.844254
7,CatBoostRegressor,0.835593
5,Random_Forest_Regressor,0.814748
6,XGBRegressor,0.787567
8,AdaBoostRegressor,0.786562
1,Lasso,0.783990
3,KNN,0.766944
4,DecisionTree,0.694864
